the whole point of this notebook is to train inception modesl with 0-0.9 label smoothing and multiple image cropping.

In [1]:
import os, glob, bcolz, gc

import numpy as np
import pandas as pd

from tqdm import tqdm
from scipy import ndimage, misc

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MinMaxScaler

from keras import backend as K
from keras import optimizers
from keras.models import Model, load_model
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, Callback
from keras.preprocessing.image import ImageDataGenerator, random_rotation, random_shear, random_zoom, random_shift, flip_axis

from keras.layers.normalization import BatchNormalization
from keras.layers import Dense, Dropout, Flatten, Activation, Input, concatenate, GlobalAveragePooling2D
from keras.layers.convolutional import MaxPooling2D, Convolution2D, AveragePooling2D
from keras.layers.advanced_activations import PReLU, LeakyReLU

from keras.applications.vgg19 import VGG19
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_v3 import InceptionV3
from keras.applications.xception import Xception
from keras.applications.inception_v3 import preprocess_input as preprocess_input_incep_xcep
from keras.applications.imagenet_utils import preprocess_input as preprocess_input_vgg_resnet

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

from tensorflow.python.client import device_lib
device_lib.list_local_devices()

Using TensorFlow backend.


[name: "/cpu:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 5945851850445322251, name: "/gpu:0"
 device_type: "GPU"
 memory_limit: 11332668621
 locality {
   bus_id: 2
 }
 incarnation: 11266887190171731157
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:85:00.0"]

In [7]:
def grab_optimizer(opt, lr):
    if opt == 'sgd':
        return optimizers.SGD(lr=lr, decay=1e-6, momentum=0.8, nesterov=True)
    elif opt == 'adam':
        return optimizers.Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

def inception_block(filter_depth, inputs):
    t1 = Convolution2D(filter_depth, (1, 1), padding='same', activation=None,use_bias=False)(inputs)
    t1 = BatchNormalization()(t1)
    t1 = Activation('relu')(t1)
    
    tower_1 = Convolution2D(filter_depth, (3, 3), padding='same', activation=None, use_bias=False)(t1)
    tower_1 = BatchNormalization()(tower_1)
    tower_1 = Activation('relu')(tower_1)
    
    tower_2 = Convolution2D(filter_depth, (5, 5), padding='same', activation=None, use_bias=False)(t1)
    tower_2 = BatchNormalization()(tower_2)
    tower_2 = Activation('relu')(tower_2)
    
    tower_3 = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(inputs)
    tower_3 = Convolution2D(filter_depth, (1, 1), padding='same', activation=None, use_bias=False)(tower_3)
    tower_3 = BatchNormalization()(tower_3)
    tower_3 = Activation('relu')(tower_3)
    
    return concatenate([tower_1, tower_2, tower_3], axis=3)

def dense_block(units, activation, drop_prob, inputs):
    x = Dense(units, activation=None)(inputs)
    x = BatchNormalization()(x)
    x = Activation(activation)(x)
    x = Dropout(drop_prob)(x)
    return x

def make_incep(input_shape, optimizer):
    inputs = Input(shape=input_shape)
    
    # convolution preproccesing
    m = BatchNormalization()(inputs)
    m = Convolution2D(16, (3, 3), strides=(2,2), padding='valid', activation='relu')(m)
    m = MaxPooling2D((3, 3), strides=(2, 2))(m)
    #inception blocks
    m = BatchNormalization()(m)
    m = inception_block(32, m)
    m = inception_block(64, m)
    m = inception_block(128, m)
    m = GlobalAveragePooling2D()(m)
    m = BatchNormalization()(m)
    m = dense_block(1024, 'relu', 0.25, inputs=m)
    m = dense_block(1024, 'relu', 0.5, inputs=m)
    outputs = dense_block(1, 'sigmoid', 0, inputs=m)
    
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

class ArocScore(Callback):
    def on_train_begin(self, logs={}):
        return
    
    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(self.validation_data[0])
        print('val aroc: {}'.format(roc_auc_score(np.around(self.validation_data[1]), y_pred)))
        
    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

# does not use precomputiation so it can use data augmentation
def train_kfolds_smoothed(model, train_data, train_label, augment, model_out, model_init_weights, epochs, kfolds, batch_size):
    kf = KFold(n_splits=kfolds, shuffle=True)
    
    i = 0
    models_stats = {}
    for train_ixs, valid_ixs in kf.split(train_data):
        x_train = train_data[train_ixs]
        x_valid = train_data[valid_ixs]
        y_train = train_label[train_ixs]
        y_valid = train_label[valid_ixs]
        
        #augment the data
        if augment:
            for i in range(len(x_train)):
                x_train[i] = random_rotation(x_train[i], 20) 
                x_train[i] = random_shear(x_train[i], 0.2)
                x_train[i] = random_zoom(x_train[i], [0.8, 1.2])
                x_train[i] = random_shift(x_train[i], 0.2, 0.2)
        
        #re-initialzie the weights of the model on each run
        #by loading thi intiial stored weights from file
        model = load_model(model_init_weights)
        model_out_file = '{}_{}.model'.format(model_out, str(i))
        model_checkpoint = ModelCheckpoint(model_out_file, 
                                            monitor='val_loss', 
                                            save_best_only=True)
        
        aroc_score = ArocScore()
        
        reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              patience=5,
                              verbose=1,
                              factor=0.1,
                              cooldown=10,
                              min_lr=0.00001)
        
        model.fit(x=x_train, y=y_train, 
                      batch_size=batch_size,
                      validation_data=(x_valid,y_valid),
                      epochs=epochs,
                      verbose=1,
                      callbacks=[aroc_score, model_checkpoint, reduce_lr])
        
        model = load_model(model_out_file)
        
        eval_tr = model.evaluate(x_train, y_train)
        eval_va = model.evaluate(x_valid, y_valid)
        
        tr_score = roc_auc_score(np.around(y_train), model.predict(x_train)[:, 0])
        va_score = roc_auc_score(np.around(y_valid), model.predict(x_valid)[:, 0])
        
        print('\n')
        print('kfold: {}'.format(str(i)))
        print('best model train loss: {}'.format(eval_tr))
        print('best model valid loss: {}'.format(eval_va))
        print('best model train aroc score: {}, valid aroc score: {}'.format(tr_score, va_score))
        print('\n')
        models_stats[model_out_file] = {'score_tr_va':[tr_score, va_score], 'train_loss':[eval_tr], 'val_loss':[eval_va]}
        
        with open(os.path.join(results_path,'{}_{}.out'.format(model_out,'history')), 'a') as f:
            f.write('kfold: {}'.format(str(i)))
            f.write('best model train loss: {}'.format(eval_tr))
            f.write('best model valid loss: {}'.format(eval_va))
            f.write('best model train aroc score: {}, valid aroc score: {}'.format(tr_score, va_score))
            f.write('\n')
        
        i += 1
    
    return models_stats

In [3]:
path = os.path.join('/scratch', 'yns207', 'data_invasive')
test_path = os.path.join(path, 'test')
results_path = os.path.join(path, 'results')
train_path = os.path.join(path, 'train')
valid_path = os.path.join(path, 'valid')

In [4]:
train_set = pd.read_csv(os.path.join(path, 'train_labels.csv'))
test_set = pd.read_csv(os.path.join(path, 'sample_submission.csv'))

def read_img(img_path, img_shape):
    img = misc.imread(img_path)
    img = misc.imresize(img, img_shape)
    return img

def read_imgs(img_height, img_width):
    train_img, test_img = [],[]
    for img_path in tqdm(train_set['name'].iloc[:]):
        train_img.append(read_img(os.path.join(path, 'train', str(img_path)+'.jpg'), (img_height, img_width)))

    for img_path in tqdm(test_set['name'].iloc[:]):
        test_img.append(read_img(os.path.join(path, 'test', str(img_path)+'.jpg'), (img_height, img_width)))
    return np.array(train_img), np.array(test_img)

train_img, test_img = read_imgs(200,300)

100%|██████████| 1531/1531 [00:53<00:00, 28.82it/s]


In [5]:
train_labels = np.array(train_set['invasive'].iloc[:])

# custom inception 200x300 0-0.9 smoothing - aug 8

In [8]:
%cd $path
batch_size = 32
epochs = 35
kfolds = 5
lr = 0.001

model_name = 'invasive_customincep200x300_aug8'
init_weights_model = '{}_base.model'.format(model_name)

model = make_incep(train_img[0].shape, grab_optimizer('adam', lr))
#recmompile the model without accuracy (since it doestn report w/ smoothed labels)
model.compile(loss='binary_crossentropy', optimizer=grab_optimizer('adam', lr))
model.save(init_weights_model)

proc_train_img = preprocess_input_incep_xcep(train_img.astype(np.float32))
smoothed_train_labels = MinMaxScaler(feature_range=(0,0.9)).fit_transform(train_labels.reshape(-1, 1)).flatten()

# train dense model on folds
performance = train_kfolds_smoothed(model, proc_train_img, smoothed_train_labels, False, model_name, init_weights_model, epochs, kfolds, batch_size)

/scratch/yns207/data_invasive


/home/yns207/anaconda3/envs/keras-py3/lib/python3.5/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, _DataConversionWarning)


Train on 1836 samples, validate on 459 samples
Epoch 1/35
1836/1836 [==============================] - 45s - loss: 0.5538 - val_loss: 0.7197
Epoch 2/35
1836/1836 [==============================] - 39s - loss: 0.4960 - val_loss: 0.7124
Epoch 3/35
1836/1836 [==============================] - 39s - loss: 0.4660 - val_loss: 0.8415
Epoch 4/35
1836/1836 [==============================] - 39s - loss: 0.4349 - val_loss: 0.8206
Epoch 5/35
1836/1836 [==============================] - 39s - loss: 0.4279 - val_loss: 0.7382
Epoch 6/35
1836/1836 [==============================] - 40s - loss: 0.4108 - val_loss: 0.5668
Epoch 7/35
1836/1836 [==============================] - 40s - loss: 0.4128 - val_loss: 0.4619
Epoch 8/35
1836/1836 [==============================] - 40s - loss: 0.3928 - val_loss: 0.4139
Epoch 9/35
1836/1836 [==============================] - 40s - loss: 0.3746 - val_loss: 0.3809
Epoch 10/35
1836/1836 [==============================] - 40s - loss: 0.3690 - val_loss: 0.3567
Epoch 11/35


In [9]:
performance

{'invasive_customincep200x300_aug8_0.model': {'score_tr_va': [0.99999233421787947,
   0.98965517241379319],
  'train_loss': [0.25847929505091605],
  'val_loss': [0.30801509713555214]},
 'invasive_customincep200x300_aug8_1.model': {'score_tr_va': [0.99962507730364869,
   0.98883296634161688],
  'train_loss': [0.23737935452419689],
  'val_loss': [0.30029142740505194]},
 'invasive_customincep200x300_aug8_2.model': {'score_tr_va': [0.99996459013076355,
   0.99329312017547566],
  'train_loss': [0.23738287710676007],
  'val_loss': [0.29335813867110833]},
 'invasive_customincep200x300_aug8_3.model': {'score_tr_va': [0.99999873315572174,
   0.99115802675585285],
  'train_loss': [0.2435891695142052],
  'val_loss': [0.30263843255884509]},
 'invasive_customincep200x300_aug8_4.model': {'score_tr_va': [0.99958223159653758,
   0.99477657620893689],
  'train_loss': [0.23652726402080138],
  'val_loss': [0.27212406224155217]}}

# custom inception 400x500 0-0.9 smoothing - aug 8

In [ ]:
train_img, test_img = read_imgs(400,500)

100%|██████████| 1531/1531 [00:58<00:00, 27.68it/s]


In [ ]:
%cd $path
batch_size = 16
epochs = 35
kfolds = 5
lr = 0.001

model_name = 'invasive_customincep400x500_aug8'
init_weights_model = '{}_base.model'.format(model_name)

model = make_incep(train_img[0].shape, grab_optimizer('adam', lr))
#recmompile the model without accuracy (since it doestn report w/ smoothed labels)
model.compile(loss='binary_crossentropy', optimizer=grab_optimizer('adam', lr))
model.save(init_weights_model)

proc_train_img = preprocess_input_incep_xcep(train_img.astype(np.float32))
smoothed_train_labels = MinMaxScaler(feature_range=(0,0.9)).fit_transform(train_labels.reshape(-1, 1)).flatten()

# train dense model on folds
performance = train_kfolds_smoothed(model, proc_train_img, smoothed_train_labels, False, model_name, init_weights_model, epochs, kfolds, batch_size)

/scratch/yns207/data_invasive


/home/yns207/anaconda3/envs/keras-py3/lib/python3.5/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, _DataConversionWarning)


Train on 1836 samples, validate on 459 samples
Epoch 1/35
1836/1836 [==============================] - 149s - loss: 0.5944 - val_loss: 0.6493
Epoch 2/35
1836/1836 [==============================] - 130s - loss: 0.5097 - val_loss: 0.8288
Epoch 3/35
1836/1836 [==============================] - 131s - loss: 0.4617 - val_loss: 0.4764
Epoch 4/35
1836/1836 [==============================] - 131s - loss: 0.4404 - val_loss: 0.4258
Epoch 5/35
1836/1836 [==============================] - 131s - loss: 0.4221 - val_loss: 0.4529
Epoch 6/35
1836/1836 [==============================] - 131s - loss: 0.3968 - val_loss: 0.3683
Epoch 7/35
1836/1836 [==============================] - 131s - loss: 0.3904 - val_loss: 0.5103
Epoch 8/35
1836/1836 [==============================] - 131s - loss: 0.3916 - val_loss: 0.3813
Epoch 9/35
1836/1836 [==============================] - 130s - loss: 0.3861 - val_loss: 0.3749
Epoch 10/35
1836/1836 [==============================] - 131s - loss: 0.3651 - val_loss: 0.3622
Ep

In [13]:
performance

{'invasive_customincep400x500_aug8_0.model': {'score_tr_va': [0.99998989449856501,
   0.99545840407470287],
  'train_loss': [0.22947887664007466],
  'val_loss': [0.29105763971675713]},
 'invasive_customincep400x500_aug8_1.model': {'score_tr_va': [0.99809521372878873,
   0.99417764214583992],
  'train_loss': [0.24604864736657775],
  'val_loss': [0.26598464079152523]},
 'invasive_customincep400x500_aug8_2.model': {'score_tr_va': [0.99904440876790135,
   0.99241085769354309],
  'train_loss': [0.25048458725538647],
  'val_loss': [0.27346646467584929]},
 'invasive_customincep400x500_aug8_3.model': {'score_tr_va': [0.99981147150975413,
   0.99245320589406605],
  'train_loss': [0.24759488414835046],
  'val_loss': [0.27529249377629855]},
 'invasive_customincep400x500_aug8_4.model': {'score_tr_va': [0.99990625352341067,
   0.9956521739130435],
  'train_loss': [0.2496488180035859],
  'val_loss': [0.28245741921052953]}}

# already have a 300x400 0-0.9 smoothed form aug 8 called invasive_customincep2_aug8